<a href="https://colab.research.google.com/github/DBCooper200/project_dl/blob/main/trying_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!ls -lha /root/.kaggle/kaggle.json

-rw------- 1 root root 67 May 23 16:59 /root/.kaggle/kaggle.json


In [3]:
!pip install -q kaggle

In [4]:
!kaggle datasets download -d himanshuagarwal1998/glaucomadataset

 99% 2.52G/2.54G [00:23<00:00, 208MB/s]
100% 2.54G/2.54G [00:23<00:00, 115MB/s]


In [5]:
!mkdir -p /content/Data_set
!unzip -q /content/glaucomadataset.zip -d /content/Data_set

In [6]:
import tensorflow 
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
import cv2
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [7]:
path = ['/content/Data_set/Non Glaucoma', '/content/Data_set/Glaucoma']
images = []
labels = []
for n,i in enumerate(path):
    for j in tqdm(os.listdir(i)):
        img_path = os.path.join(i,j)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64,64))
        images.append(img)
        labels.append(n)
images = np.array(images)/255
labels = np.array(labels)

100%|██████████| 511/511 [00:09<00:00, 52.00it/s]


In [8]:
for n in enumerate(path):
  print(n)

#the labels assigned are 
# 0 for non glaucomic images
# 1 for glaucomic images

(0, '/content/Data_set/Non Glaucoma')
(1, '/content/Data_set/Glaucoma')


In [9]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, labels = shuffle(images, labels, random_state=32)
x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.15, random_state=44)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=40)
     

In [10]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(zoom_range=0.2, horizontal_flip=True, vertical_flip=True,
                            fill_mode='constant', cval=0.)
train_gen = datagen.flow(x_train, y_train, batch_size=32)
#Using TensorFlow backend.

#for test set
test_gen = datagen.flow(x_test, y_test, batch_size = 32)

In [ ]:
initializer = tf.keras.initializers.HeNormal()

In [35]:
model = Sequential()
model.add(Flatten())
model.add(Dense(256, input_shape=(64,64), activation='relu',kernel_initializer=initializer))
model.add(Dense(128, activation='relu',kernel_initializer=initializer))
#model.add(Dense(333, activation='relu',kernel_initializer=initializer))
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
tb_callback = tensorflow.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)
#the above is done to get tensorboard logs which are then referred by callback parameter below
model.fit_generator(train_gen, epochs=10, steps_per_epoch=1,verbose=1, validation_data=(x_valid,y_valid),callbacks=[tb_callback])
'''
model.fit(x_train, y_train,steps_per_epoch=1, epochs=100,verbose="auto",callbacks=[tb_callback],validation_data=(x_valid,y_valid))                              
model.summary()
'''

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.8818 - accuracy: 0.6250 - val_loss: 4.8816 - val_accuracy: 0.5390
Epoch 2/10
1/1 [==============================] - 1s 586ms/step - loss: 4.5018 - accuracy: 0.5938 - val_loss: 1.1064 - val_accuracy: 0.5390
Epoch 3/10
1/1 [==============================] - 1s 567ms/step - loss: 0.9284 - accuracy: 0.5938 - val_loss: 1.2561 - val_accuracy: 0.5390
Epoch 4/10
1/1 [==============================] - 1s 540ms/step - loss: 1.5398 - accuracy: 0.5625 - val_loss: 0.8637 - val_accuracy: 0.5390
Epoch 5/10
1/1 [==============================] - 1s 560ms/step - loss: 1.7070 - accuracy: 0.6562 - val_loss: 0.4370 - val_accuracy: 0.5390
Epoch 6/10
1/1 [==============================] - 1s 584ms/step - loss: 0.4897 - accuracy: 0.3438 - val_loss: 0.6663 - val_accuracy: 0.5390
Epoch 7/10
1/1 [==============================] - 1s 574ms/step - loss: 0.7995 - accuracy: 0.4375 - val_loss: 0.5131 - val_accuracy: 0.5390
Epoch 8/10
1/1 [=======

'\nmodel.fit(x_train, y_train,steps_per_epoch=1, epochs=100,verbose="auto",callbacks=[tb_callback],validation_data=(x_valid,y_valid))                              \nmodel.summary()\n'

In [ ]:
tb_callback = tensorflow.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)
#the above is done to get tensorboard logs which are then referred by callback parameter below
history = classifier.fit_generator(train_gen, epochs=100, steps_per_epoch=1,
                              verbose=1, validation_data=(x_valid,y_valid),
                              callbacks=[tb_callback])
#Number of Steps per Epoch = (Total Number of Training Samples) / (Batch Size)
#callbacks reqd as logs were needed and manual saving of model will be done furthur to the disk